### Import Knet & ONNX

In [1]:
using Knet;

In [2]:
KnetOnnx_PATH = "/Users/syny/kafamagore/KnetOnnx.jl/src"
push!(LOAD_PATH, KnetOnnx_PATH)
using KnetOnnx;

In [3]:
include(Knet.dir("data","mnist.jl"))
dtrn,dtst = mnistdata();

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Pkg.jl:531
┌ Info: Loading MNIST...
└ @ Main /Users/syny/.julia/packages/Knet/bTNMd/data/mnist.jl:33


### Read ONNX Graph

In [4]:
file_path = "cnn.onnx"
graph = ONNXtoGraph(file_path);

In [5]:
PrintGraph(graph)

model inputs: ["input.1"]
model outputs: ["16"]
(op1) Conv
	input1: input.1
	input2: conv1.weight
	input3: conv1.bias
	output1: 9
(op2) Relu
	input1: 9
	output1: 10
(op3) Conv
	input1: 10
	input2: conv2.weight
	input3: conv2.bias
	output1: 11
(op4) MaxPool
	input1: 11
	output1: 12
(op5) Flatten
	input1: 12
	output1: 13
(op6) Gemm
	input1: 13
	input2: fc1.weight
	input3: fc1.bias
	output1: 14
(op7) Relu
	input1: 14
	output1: 15
(op8) Gemm
	input1: 15
	input2: fc2.weight
	input3: fc2.bias
	output1: 16


### Re-construct model in Knet

In [6]:
model = KnetModel(file_path);

In [7]:
model.tensors

Dict{Any,Any} with 9 entries:
  "16"      => Nothing
  "12"      => Nothing
  "10"      => Nothing
  "15"      => Nothing
  "input.1" => Nothing
  "9"       => Nothing
  "11"      => Nothing
  "13"      => Nothing
  "14"      => Nothing

In [8]:
x,y = first(dtrn)
model(x)

10×100 Array{Float64,2}:
  0.122901     0.129133     0.12323     …   0.129186     0.11251
  0.0393511    0.0247273    0.0446674       0.0515631    0.0369746
  0.00084469   0.0200402    0.015049        0.0133032    0.0115262
  0.0839118    0.0726088    0.0726914       0.0872949    0.0713523
 -0.0454412   -0.0211448   -0.00398575     -0.0472456   -0.0228458
 -0.0982851   -0.0789427   -0.0745983   …  -0.0706251   -0.0759159
  0.0142066    0.00980528  -0.00219376      0.00719757  -0.00854764
 -0.00444491  -0.00343109  -0.0101027      -0.0205154   -0.0251139
 -0.0650283   -0.0685512   -0.0562406      -0.0774696   -0.0687056
  0.0790243    0.0754361    0.0773273       0.0897365    0.0757764

In [9]:
model.tensors

Dict{Any,Any} with 9 entries:
  "16"      => [0.122901 0.129133 … 0.129186 0.11251; 0.0393511 0.0247273 … 0.0…
  "12"      => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.116799; … ; 0.0 0.0 ……
  "10"      => Float32[0.238287 0.238287 … 0.238287 0.238287; 0.238287 0.238287…
  "15"      => [0.0822569 0.0 … 0.0906895 0.0999589; 0.0 0.0 … 0.0 0.0; … ; 0.0…
  "input.1" => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …
  "9"       => Float32[0.238287 0.238287 … 0.238287 0.238287; 0.238287 0.238287…
  "11"      => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …
  "13"      => Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …
  "14"      => [0.0822569 -0.00749586 … 0.0906895 0.0999589; -0.0728656 -0.0673…

In [10]:
@show accuracy(model, dtst)

accuracy(model, dtst) = 0.1136


0.1136

### Re-train the model in Knet

In [11]:
EPOCH = 2
progress!(sgd(model, repeat(dtrn,EPOCH)))
@show accuracy(model, dtst)

┌ Warning: repeat(d::Data,n) is deprecated, use IterTools.ncycle instead.
└ @ Knet /Users/syny/.julia/packages/Knet/bTNMd/src/data.jl:92



Stacktrace:
 [1] getindex(::Array{String,1}, ::Int64) at ./array.jl:788
 [2] (::KnetModel)(::Array{Float32,4}, ::Vararg{Any,N} where N) at /Users/syny/kafamagore/KnetOnnx.jl/src/KnetModel.jl:121
 [3] (::Knet.var"#693#694"{Knet.Minimize{Knet.Repeat},Tuple{Array{Float32,4},Array{UInt8,1}}})() at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:205
 [4] differentiate(::Function; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:144
 [5] differentiate at /Users/syny/.julia/packages/AutoGrad/6QsMu/src/core.jl:135 [inlined]
 [6] iterate at /Users/syny/.julia/packages/Knet/bTNMd/src/train.jl:23 [inlined]
 [7] iterate at /Users/syny/.julia/packages/Knet/bTNMd/src/progress.jl:69 [inlined]
 [8] progress!(::Knet.Minimize{Knet.Repeat}; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /Users/syny/.julia/packages/Knet/bTNMd/src/progress.jl:58
 [9] progress!(::Knet.Minimize{Knet.Repeat})

BoundsError: BoundsError: attempt to access 1-element Array{String,1} at index [2]

In [ ]:
using Images, Plots
function predictMNIST(model, data)
    i = rand(1:100)
    x = reshape(rand(dtrn)[1][:,:,:,i], 28, 28, 1, 1)
    pred = argmax(model(x), dims=1)[1][1]; if pred==10; pred=0; end
    image = x[:,:,1,1]    
    plot(heatmap(permutedims(image), yflip = true, title = ("Model Output: " * string(pred))))
end
predictMNIST(model, dtst)